In [ ]:
import numpy as np
from collections import Counter, defaultdict
import concurrent.futures

# Define ranks and suits
ranks = np.arange(13)  # Map ranks to integers 0-12 (2 to Ace)
suits = np.arange(4)   # Map suits to integers 0-3 (hearts, diamonds, clubs, spades)

# Create a deck with two identical sets of 52 cards
deck = np.array([(rank, suit) for rank in ranks for suit in suits] * 2, dtype=[('rank', 'i4'), ('suit', 'i4')])

def print_deck(deck):
    """Prints the entire deck in a readable format."""
    deck_cards = []
    for card in deck:
        rank = card['rank']
        suit = card['suit']
        suit_name = ["Hearts", "Diamonds", "Clubs", "Spades"][suit]
        rank_name = ["2", "3", "4", "5", "6", "7", "8", "9", "10", "J", "Q", "K", "A"][rank]
        deck_cards.append(f"{rank_name} of {suit_name}")
    print(", ".join(deck_cards))

def print_hand(hand):
    """Print the hand in a readable 'rank of suit' format, sorted by three-of-a-kind first, then pair."""
    rank_counts = Counter([card['rank'] for card in hand])
    # Identify the three-of-a-kind rank and the pair rank
    three_of_a_kind_rank = next((rank for rank, count in rank_counts.items() if count == 3), None)
    pair_rank = next((rank for rank, count in rank_counts.items() if count == 2), None)

    # Sort hand by three-of-a-kind cards first, then pair
    sorted_hand = sorted(hand, key=lambda card: (card['rank'] != three_of_a_kind_rank, card['rank'] != pair_rank))

    hand_cards = []
    for card in sorted_hand:
        rank = card['rank']
        suit = card['suit']
        suit_name = ["Hearts", "Diamonds", "Clubs", "Spades"][suit]
        rank_name = ["2", "3", "4", "5", "6", "7", "8", "9", "10", "J", "Q", "K", "A"][rank]
        hand_cards.append(f"{rank_name} of {suit_name}")
    print("Hand:", ", ".join(hand_cards))

def deal_hand(deck):
    """
    Shuffle the deck and select five unique cards.
    """
    shuffled_deck = np.random.permutation(deck)
    hand_indices = np.random.choice(len(shuffled_deck), 5, replace=False)
    hand = shuffled_deck[hand_indices]
    sorted_hand = np.sort(hand, order='rank')
    return shuffled_deck, sorted_hand

def check_full_house_case(hand):
    rank_counts = Counter([card['rank'] for card in hand])

    # Find the three-of-a-kind rank
    three_of_a_kind_rank = None
    pair_rank = None

    for rank, count in rank_counts.items():
        if count == 3:
            three_of_a_kind_rank = rank
        elif count == 2:
            pair_rank = rank

    # A valid Full House must have both a three-of-a-kind and a pair
    if not three_of_a_kind_rank or not pair_rank:
        return -1  # Not a Full House

    # Separate the three-of-a-kind and pair cards for suit checks
    three_of_a_kind_cards = [card for card in hand if card['rank'] == three_of_a_kind_rank]
    pair_cards = [card for card in hand if card['rank'] == pair_rank]

    # Case 0: Three cards of the same rank with different suits; pair cards can be any suit
    if (
        len(set(card['suit'] for card in three_of_a_kind_cards)) == 3  # Three unique suits for three-of-a-kind
        and len(set(card['suit'] for card in pair_cards)) == 2  # Distinct suits for the pair
    ):
        return 0

    # Case 1:
    # Option 1: Three-of-a-kind has two identical suits and a third different suit, and the pair cards have different suits
    if (
        len(set(card['suit'] for card in three_of_a_kind_cards)) == 2  # Three-of-a-kind has two identical suits
        and len(set(card['suit'] for card in pair_cards)) == 2  # Pair cards have different suits
    ):
        return 10  # Standard Case 1

    # Option 2: Three-of-a-kind has all different suits, and pair cards have the same suit (Case 1, Option 2)
    if (
        len(set(card['suit'] for card in three_of_a_kind_cards)) == 3  # Three-of-a-kind has three unique suits
        and len(set(card['suit'] for card in pair_cards)) == 1  # Pair cards have the same suit
    ):
        return 11  # Use '10' to denote Case 1, Option 2 specifically

    # Case 2: Three-of-a-kind has two identical suits and a third different suit,
    # and the pair cards are of the same suit
    if (
        len(set(card['suit'] for card in three_of_a_kind_cards)) == 2  # Three-of-a-kind has two identical suits
        and len(set(card['suit'] for card in pair_cards)) == 1  # Pair cards have the same suit
    ):
        return 2

    return -1  # If none of the cases match

def run_simulation(num_trials, worker_id=None):
    """
    Run simulation with enhanced random seed handling and batching.
    Counts Full House cases and stores example hands for each case.
    """
    if worker_id is not None:
        # Create a unique seed using large prime numbers and worker id
        base_seed = 104729  # Large prime
        multiplier = 15485863  # Another large prime
        worker_seed = base_seed + (worker_id * multiplier)
        np.random.seed(worker_seed)

    counts = Counter()
    local_examples = defaultdict(list)  # Store examples for each case

    # Process hands in smaller batches
    batch_size = 100_000
    num_batches = num_trials // batch_size + (1 if num_trials % batch_size else 0)

    for batch in range(num_batches):
        current_batch_size = min(batch_size, num_trials - batch * batch_size)
        batch_counts = Counter()

        for _ in range(current_batch_size):
            _, hand = deal_hand(deck)
            case = check_full_house_case(hand)

            if case in [0, 10, 2, 11]:  # Valid cases including Case 1, Option 2
                batch_counts[case] += 1
                # Collect examples based on case
                if case == 10 or case == 11:  # Collect all for Case 1 and Case 1 Option 2
                    local_examples[case].append(hand.copy())
                elif len(local_examples[case]) < 10:  # Limit to 10 for others
                    local_examples[case].append(hand.copy())

        # Update total counts
        counts.update(batch_counts)

    return counts, local_examples

def run_simulation_parallel(trials, num_workers=16):
    """
    Run parallel simulation with improved worker management and progress tracking.
    Combines counts and example hands from all workers.
    """
    chunk_size = trials // num_workers
    remaining = trials % num_workers
    total_counts = Counter()
    all_examples = defaultdict(list)

    with concurrent.futures.ProcessPoolExecutor(max_workers=num_workers) as executor:
        # Distribute work with adjusted chunks
        futures = []
        for i in range(num_workers):
            # Last worker gets remaining hands if any
            worker_trials = chunk_size + (remaining if i == num_workers-1 else 0)
            futures.append(executor.submit(run_simulation, worker_trials, worker_id=i))

        # Track progress and collect results
        results = []
        for future in concurrent.futures.as_completed(futures):
            counts, examples = future.result()
            results.append((counts, examples))

    # Combine all results
    for counts, examples in results:
        total_counts.update(counts)
        # Merge examples based on case
        for case in [0, 10, 2, 11]:
            if case in [10, 11]:  # Collect all examples for Case 1 and Case 1 Option 2
                all_examples[case].extend(examples[case])
            else:  # Limit to 10 examples for Case 0 and Case 2
                remaining_slots = 10 - len(all_examples[case])
                if remaining_slots > 0:
                    all_examples[case].extend(examples[case][:remaining_slots])

    return total_counts, all_examples

def calculate_total_full_house_probability(counts, total_hands):
    """Calculate the total probability of getting a Full House including all cases."""
    # Calculate total for Case 0
    case0_total = counts[0]

    # Calculate total for Case 1 (both options)
    case1_total = counts[10] + counts[11]  # Standard Case 1 + Case 1 Option 2

    # Calculate total for Case 2
    case2_total = counts[2]

    # Sum all cases
    total_full_house = case0_total + case1_total + case2_total

    # Calculate individual probabilities

    total_prob = total_full_house / total_hands


    print(f"Total Full House: {total_full_house} occurrences, probability = {total_prob:.8f}")

    return total_prob

def main():
    # Print the initial unshuffled deck
    print("Initial deck:")
    print_deck(deck)

    # Part 1: Run 100 trials, shuffle each time, and print the shuffled deck and dealt hand
    print("\nRunning 100 trials:")
    example_hands_100 = {"Case 0": [], "Case 1": [], "Case 2": []}
    for trial in range(100):
        shuffled_deck, hand = deal_hand(deck)  # Shuffle and deal a hand
        print(f"\nTrial {trial + 1}: Shuffled deck")
        print_deck(shuffled_deck)  # Print the entire shuffled deck after shuffling

        # Print the first 5 cards as the dealt hand
        print_hand(hand)

        # Check for a four-of-a-kind in the dealt hand
        case = check_full_house_case(hand)
        print(f"Full_House_Case: {case}")

        # Collect examples during the initial 100 trials if needed
        if case in example_hands_100 and len(example_hands_100[case]) < 10:
            example_hands_100[case].append(hand)

    # Part 2: Run full simulation for larger trial counts and calculate total four-of-a-kind probability
    trial_counts = [100, 1_000, 10_000, 100_000, 1_000_000, 10_000_000, 100_000_000]
    for trials in trial_counts:
      print(f"\nRunning parallel simulation with {trials} trials...")
      counts, example_hands = run_simulation_parallel(trials)

    # Calculate total probability
      total_probability = calculate_total_full_house_probability(counts, trials)  # Pass 'trials' as total_hands

      print(f"Results for {trials} trials:")
      print(f"Case 0 (all different suits): {counts[0]}")
      print(f"Case 1 (two same suit): {counts[10]}")
      print(f"Case 2 (two pairs same suit): {counts[2]}")
      print(f"Total probability: {total_probability:.8f}")


        # Print examples for 1,000,000 trial run
      if trials == 1_000_000:
            print("\nExample hands from 1,000,000 trials:")
            for case in ["Case 0", "Case 1", "Case 2"]:
                print(f"\n{case} examples:")
                for hand in example_hands[case]:
                    print_hand(hand)

if __name__ == "__main__":
    main()


Initial deck:
2 of Hearts, 2 of Diamonds, 2 of Clubs, 2 of Spades, 3 of Hearts, 3 of Diamonds, 3 of Clubs, 3 of Spades, 4 of Hearts, 4 of Diamonds, 4 of Clubs, 4 of Spades, 5 of Hearts, 5 of Diamonds, 5 of Clubs, 5 of Spades, 6 of Hearts, 6 of Diamonds, 6 of Clubs, 6 of Spades, 7 of Hearts, 7 of Diamonds, 7 of Clubs, 7 of Spades, 8 of Hearts, 8 of Diamonds, 8 of Clubs, 8 of Spades, 9 of Hearts, 9 of Diamonds, 9 of Clubs, 9 of Spades, 10 of Hearts, 10 of Diamonds, 10 of Clubs, 10 of Spades, J of Hearts, J of Diamonds, J of Clubs, J of Spades, Q of Hearts, Q of Diamonds, Q of Clubs, Q of Spades, K of Hearts, K of Diamonds, K of Clubs, K of Spades, A of Hearts, A of Diamonds, A of Clubs, A of Spades, 2 of Hearts, 2 of Diamonds, 2 of Clubs, 2 of Spades, 3 of Hearts, 3 of Diamonds, 3 of Clubs, 3 of Spades, 4 of Hearts, 4 of Diamonds, 4 of Clubs, 4 of Spades, 5 of Hearts, 5 of Diamonds, 5 of Clubs, 5 of Spades, 6 of Hearts, 6 of Diamonds, 6 of Clubs, 6 of Spades, 7 of Hearts, 7 of Diamonds, 